In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Encontrar todas as marcas
url = 'https://www.drogariasminasmais.com.br/medicamentos'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

marcas = soup.find_all("label", class_="vtex-checkbox__label w-100 c-on-base pointer")
marcas_corr = []
for marca in marcas:
    marcas_corr.append(str(marca).split('>')[1].split('<')[0])
for i in range(len(marcas_corr)):
    marcas_corr[i] = marcas_corr[i].lower().replace(" ", '-')
marcas_corr = marcas_corr[19:]

print(marcas_corr)

['hypera', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'cimed', 'germed-generico', 'libbs', 'apsen', 'biolab-rx', 'supera', 'medley-genericos', 'momenta', 'farmoquimica', 'eurofarma-generico', 'myralis', 'abbott-rx', 'teuto-generico', 'ache-otc', 'sanofi-rx', 'cristalia/latinofarma', 'sanofi-otc', 'prati-donaduzzi-generico', 'torrent', 'legrand-generico', 'biolab-generico', 'uniao-quimica-similar', 'neo-quimica-generico', 'uniao-quimica-genom', 'pharlab', 'neo-quimica-similar', 'nova-quimica-generico', 'geolab-generico', 'gsk-otc', 'marjan', 'novartis-rx', 'zanphy', 'gsk-rx', 'uniao-quimica-generico', 'procter--gambl', 'biolab-avert-otc', 'merck-rx', 'alcon', 'biosintetica-generico', 'servier', 'allergan', 'teuto-similar', 'ems-similar', 'bayer-rx', 'globo-generico', 'cellera', 'astrazeneca', 'sandoz-generico', 'herbarium', 'divcom', 'airela', 'zodiac-adium', 'sankyo-pharma', 'ranbaxy', 'natulab', 'bayer-otc', 'takeda', 'legrand-rx', 'boehringer', 'sigma-pharma-otc', 

In [8]:
def extrair_eans(string):
    eans = []  # Lista para armazenar os valores de EAN
    
    # Encontrar a posição da primeira ocorrência de '"ean"'
    indice_ean = string.find('"ean"')
    
    # Enquanto houver ocorrências de '"ean"'
    while indice_ean != -1:
        # Pegar os 13 caracteres após a posição da ocorrência de '"ean"'
        ean = string[indice_ean + len('"ean"')+2:indice_ean + len('"ean"')+2+13]
        eans.append(ean)
        
        # Encontrar a próxima ocorrência de '"ean"' após a posição atual
        indice_ean = string.find('"ean"', indice_ean + 1)
    
    return eans

# Exemplo de uso
texto = '... "ean":"1234567890123", "ean":"9876543210987", ...'
print(extrair_eans(texto))  # Saída: ['1234567890123', '9876543210987']



['1234567890123', '9876543210987']


In [9]:
# "productReference" = sku

def extrair_product_references(string):
    product_references = []  # Lista para armazenar os valores de productReference
    
    # Encontrar a posição da primeira ocorrência de '"productReference"'
    indice_reference = string.find('"productReference"')
    
    # Enquanto houver ocorrências de '"productReference"'
    while indice_reference != -1:
        # Encontrar o índice do próximo caractere após a ocorrência de '"productReference"'
        indice_inicio_valor = string.find(':', indice_reference) + 2
        
        # Encontrar o índice do próximo caractere não numérico após o início do valor
        indice_fim_valor = indice_inicio_valor
        while indice_fim_valor < len(string) and string[indice_fim_valor].isdigit():
            indice_fim_valor += 1
        
        # Pegar o valor entre os índices de início e fim
        product_reference = string[indice_inicio_valor:indice_fim_valor]
        product_references.append(product_reference)
        
        # Encontrar a próxima ocorrência de '"productReference"' após a posição atual
        indice_reference = string.find('"productReference"', indice_fim_valor)
    
    return product_references

# Exemplo de uso
texto = '... "productReference":"123", "productReference":"45689", ...'
print(extrair_product_references(texto))  # Saída: ['123', '45689']

#29595

['123', '45689']


In [10]:
def extrair_product_names(string):
    product_names = []  # Lista para armazenar os valores de productName
    
    # Encontrar a posição da primeira ocorrência de '"productName":'
    indice_name = string.find('"productName":')
    
    # Enquanto houver ocorrências de '"productName":'
    while indice_name != -1:
        # Encontrar o índice do próximo caractere após a ocorrência de '"productName":'
        indice_inicio_nome = indice_name + len('"productName":') + 1
        
        # Encontrar o índice do próximo caractere que indica o final do nome
        indice_fim_nome = string.find('"', indice_inicio_nome)
        
        # Pegar o valor entre os índices de início e fim
        product_name = string[indice_inicio_nome:indice_fim_nome]
        product_names.append(product_name)
        
        # Encontrar a próxima ocorrência de '"productName":' após a posição atual
        indice_name = string.find('"productName":', indice_fim_nome)
    
    return product_names

# Exemplo de uso
texto = '... "productName":"Produto A", "productName":"Produto B", ...'
print(extrair_product_names(texto))  # Saída: ['Produto A', 'Produto B']


['Produto A', 'Produto B']


In [11]:
def extrair_precos(string):
    precos = []  # Lista para armazenar os valores de preço
    
    # Encontrar a posição da primeira ocorrência de '"Price":'
    indice_preco = string.find('"Price":')
    
    # Enquanto houver ocorrências de '"Price":'
    while indice_preco != -1:
        # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
        indice_inicio_preco = indice_preco + len('"Price":') + 0
        
        # Encontrar o índice do próximo caractere que indica o final do preço
        indice_fim_preco = indice_inicio_preco
        while string[indice_fim_preco].isdigit() or string[indice_fim_preco] == '.':
            indice_fim_preco += 1
        
        # Pegar o valor entre os índices de início e fim
        preco = string[indice_inicio_preco:indice_fim_preco]
        precos.append(float(preco))
        
        # Encontrar a próxima ocorrência de '"Price":' após a posição atual
        indice_preco = string.find('"Price":', indice_fim_preco)
    
    return precos

# Exemplo de uso
texto = '... "Price":10.5, "Price":20.75, ...'
print(extrair_precos(texto))  # Saída: [10.5, 20.75]


[10.5, 20.75]


In [12]:
for marca in marcas_corr:
    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1'
    print(url)

    # fazendo a requisição com o site 
    requisicao = requests.get(url)

    ean_lista = extrair_eans(requisicao.text)
    sku_lista = extrair_product_references(requisicao.text)
    nome_lista = extrair_product_names(requisicao.text)
    preco_lista = extrair_precos(requisicao.text)

    print(preco_lista)

https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
[84.73, 64.59, 108.27, 54.71, 61.52, 97.15, 51.18, 59.73, 38.24, 10.57, 40.82, 22.01, 15.66, 54.99, 65.17]
https://www.drogariasminasmais.com.br/medicamentos/ache-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1


: 